In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

In [2]:
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor, LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
seed=42
np.random.seed(seed)


In [3]:
train_path = "/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"
train_data = pd.read_csv(train_path)
train_data

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


## Cleaning 

1. drop column have same values or 90%
2. drop column dont have any meaning or not found on test ["Id", "EnglishRules", "LudRules", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"]

In [4]:
# select columns with 90% or more identical values and columns with all null values
def drop_column_name(df):
    drop_columns = []
    for column in df.columns:
        if df[column].value_counts().max() / len(df) >= 0.90 or df[column].isnull().all():
            drop_columns.append(column)
    return drop_columns

In [5]:
drop_column = drop_column_name(train_data)
#add Column dont have any meaning whene training or not found on test dataset
drop_column.extend(["Id", "EnglishRules", "LudRules", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"])

In [6]:
train = train_data.drop(columns=drop_column, errors='ignore')

In [7]:
train

,GameRulesetName,agent1,agent2,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Shape,SquareShape,RegularShape,...,SetSiteState,Variable,SetVar,CopyContext,Then,ForEachPiece,DoLudeme,PlayoutsPerSecond,MovesPerSecond,utility_agent1
0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,298.07,18877.17,-0.466667
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,298.07,18877.17,-0.333333
2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,298.07,18877.17,-0.066667
3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,298.07,18877.17,-0.333333
4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,298.07,18877.17,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,0,0,0,0,1,1,1,...,0,0,0,0,1,1,0,157.52,157174.58,-0.733333
233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,0,0,0,0,1,1,1,...,0,0,0,0,1,1,0,157.52,157174.58,0.266667
233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,1,1,1,...,0,0,0,0,1,1,0,157.52,157174.58,0.666667
233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,0,0,0,0,1,1,1,...,0,0,0,0,1,1,0,157.52,157174.58,0.666667


In [9]:
train_v1=train.copy()

## Feature Engineering 

Extract feature from agent1 & agent2 

In [13]:
train_v1[['agent1_format','agent1_strategies', 'agent1_EXP_const', 'agent1_PLAYOUT','agent1_SCORE_BOUNDS']] = train_v1['agent1'].str.split('-', expand=True)
train_v1[['agent2_format','agent2_strategies', 'agent2_EXP_const', 'agent2_PLAYOUT','agent2_SCORE_BOUNDS']] = train_v1['agent2'].str.split('-', expand=True)

In [14]:
train_v1 = train_v1.drop(columns=['agent1_format','agent2_format','agent1','agent2'], errors='ignore')

In [15]:
train_v1

,GameRulesetName,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Shape,SquareShape,RegularShape,PolygonShape,Tiling,...,utility_agent1,sd,agent1_strategies,agent1_EXP_const,agent1_PLAYOUT,agent1_SCORE_BOUNDS,agent2_strategies,agent2_EXP_const,agent2_PLAYOUT,agent2_SCORE_BOUNDS
0,00Y,0,0,0,0,1,0,0,1,1,...,-0.466667,MCTS,ProgressiveHistory,0.1,MAST,false,ProgressiveHistory,0.6,Random200,false
1,00Y,0,0,0,0,1,0,0,1,1,...,-0.333333,MCTS,ProgressiveHistory,0.1,MAST,false,UCB1GRAVE,0.6,NST,true
2,00Y,0,0,0,0,1,0,0,1,1,...,-0.066667,MCTS,ProgressiveHistory,0.1,MAST,true,UCB1,0.1,NST,false
3,00Y,0,0,0,0,1,0,0,1,1,...,-0.333333,MCTS,ProgressiveHistory,0.1,MAST,true,UCB1,0.6,NST,false
4,00Y,0,0,0,0,1,0,0,1,1,...,-0.333333,MCTS,ProgressiveHistory,0.1,MAST,true,UCB1GRAVE,1.41421356237,NST,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,Zuz_Mel_7x7,0,0,0,0,1,1,1,1,1,...,-0.733333,MCTS,UCB1Tuned,1.41421356237,NST,false,ProgressiveHistory,1.41421356237,Random200,false
233230,Zuz_Mel_7x7,0,0,0,0,1,1,1,1,1,...,0.266667,MCTS,UCB1Tuned,1.41421356237,Random200,false,UCB1,0.6,MAST,false
233231,Zuz_Mel_7x7,0,0,0,0,1,1,1,1,1,...,0.666667,MCTS,UCB1Tuned,1.41421356237,Random200,false,UCB1GRAVE,1.41421356237,NST,false
233232,Zuz_Mel_7x7,0,0,0,0,1,1,1,1,1,...,0.666667,MCTS,UCB1Tuned,1.41421356237,Random200,false,UCB1GRAVE,1.41421356237,NST,true


In [17]:
def summarize_dataframe_info(df):
    table=pd.DataFrame(columns=["Column","Data Type","Null count"])
    Dtype=""
    null=0
    for column in df.columns:
        Dtype=df[column].dtype
        null=df[column].isnull().sum()
        new_row = {"Column": column, "Data Type": Dtype,"Null count":null}
        table = pd.concat([table, pd.DataFrame([new_row])], ignore_index=True)
    return table

In [18]:
info=summarize_dataframe_info(train_v1)

In [19]:
info

,Column,Data Type,Null count
0,GameRulesetName,object,0
1,Asymmetric,int64,0
2,AsymmetricForces,int64,0
3,AsymmetricPiecesType,int64,0
4,PlayersWithDirections,int64,0
...,...,...,...
261,agent1_SCORE_BOUNDS,object,0
262,agent2_strategies,object,0
263,agent2_EXP_const,object,0
264,agent2_PLAYOUT,object,0


In [27]:
print(info[info['Null count']>0])

Empty DataFrame
Columns: [Column, Data Type, Null count]
Index: []


In [26]:
print(info[info['Data Type']=='object'])

                  Column Data Type Null count
0        GameRulesetName    object          0
257                   sd    object          0
258    agent1_strategies    object          0
259     agent1_EXP_const    object          0
260       agent1_PLAYOUT    object          0
261  agent1_SCORE_BOUNDS    object          0
262    agent2_strategies    object          0
263     agent2_EXP_const    object          0
264       agent2_PLAYOUT    object          0
265  agent2_SCORE_BOUNDS    object          0


In [28]:
# Example data
data = ['Red', 'Blue', 'Green', 'Blue']

# Initialize label encoder
encoder = LabelEncoder()

# Fit and transform the data
encoded_data = encoder.fit_transform(data)

print(encoded_data)

[2 0 1 0]


In [32]:

# Example data
train_data = np.array(['Red', 'Blue', 'Green'])
test_data = np.array(['Red', 'ss', 'Green'])  # 'ss' is unseen

# Initialize label encoder
encoder = LabelEncoder()

# Fit the encoder on the training data
encoder.fit(train_data)

# Custom transformation function
def transform_with_custom_values(encoder, new_labels, default_value='unknown'):
    # Create an array for the transformed labels
    transformed = []
    
    for label in new_labels:
        if label in encoder.classes_:
            transformed.append(encoder.transform([label])[0])  # Use encoder to transform known labels
        else:
            transformed.append(default_value)  # Replace unseen labels with the default value
            
    return np.array(transformed)

# Transform the test data with custom handling for unseen labels
test_encoded = transform_with_custom_values(encoder, test_data, default_value='999')  # Assign '999' for unseen labels

print("Transformed test data:", test_encoded)

Transformed test data: ['2' '999' '1']
